In [1]:
import os
import pandas as pd
import re  # 正規表現モジュール
import requests
from bs4 import BeautifulSoup
!pip install lxml  # htmlのperserライブラリ
!pip install --upgrade beautifulsoup4 lxml

# !pip install html5lib
# !pip install --upgrade beautifulsoup4 html5lib

# !pip show lxml
# !pip show html5lib

#現在のディレクトリを確認
current_directory = os.getcwd()
print("Current working directory:", current_directory)


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Current working directory: /Users/sawad/dev/myapps/u-on-zu_data-processing


In [2]:
# 地域ページ内の国リンク配列を取得する関数
def fetch_country_links(url):
    response = requests.get(url)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    country_links = soup.find_all(href=re.compile("StationList.php"))

    return country_links


In [3]:
# 国ページ内の都市リンク配列を取得する関数
def fetch_city_links(url):
    response = requests.get(url)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    city_links = soup.find_all(href=re.compile("NrmMonth.php"))

    return city_links

In [12]:
# 緯度・経度・高度を抽出し,目的データに変換する関数
## 引数はtagからtext形式に変換しておくこと

# 緯度
def get_latitude(text):
    lat_pattern = r"緯度：([\d.]+)°([NS])"
    lat_match = re.search(lat_pattern, text)
    if lat_match:
        lat_value = float(lat_match.group(1))   # キャプチャグループ1は ([\d.]+) 部分
        lat_direction = lat_match.group(2)      # キャプチャグループ2は ([NS]) 部分
        
        if lat_direction == 'S':                # 南半球Sなら値を負数に
            latitude = -lat_value
        else:
            latitude = lat_value

    return latitude


# 経度
def get_longitude(text):
    lon_pattern = r"経度：([\d.]+)°([EW])"
    lon_match = re.search(lon_pattern, text)
    if lon_match:
        lon_value = float(lon_match.group(1))
        lon_direction = lon_match.group(2)
        
        if lon_direction == 'W':
            longitude = -lon_value
        else:
            longitude = lon_value

    return longitude


# 高度
def get_altitude(text):
    # 高度を抽出
    alt_pattern = r"高度：(\d+) \(m\)"
    alt_match = re.search(alt_pattern, text)

    # 高度情報が - となっている場合があるので初期化
    altitude = "-"
    
    if alt_match:
        altitude = int(alt_match.group(1))

    return altitude


In [13]:
# 最終出力データフレーム
new_df = pd.DataFrame(columns=range(0,32))

# ヘッダーを追加
header = ['country_code', 'country_name', 'city_code', 'city_name',  
          'latitude', 'longitude', 'altitude',
          'temp_jan', 'temp_feb', 'temp_mar', 'temp_apr', 'temp_may', 'temp_jun',
          'temp_jul', 'temp_aug', 'temp_sep', 'temp_oct', 'temp_nov', 'temp_dec',
          'rain_jan', 'rain_feb', 'rain_mar', 'rain_apr', 'rain_may', 'rain_jun',
          'rain_jul', 'rain_aug', 'rain_sep', 'rain_oct', 'rain_nov', 'rain_dec',
          'url']
new_df.loc[0] = header

In [14]:
# ベースURL
base_url = "https://www.data.jma.go.jp/cpd/monitor/nrmlist/"

# 地域コードのリスト
region_codes = ["01", "02", "03", "04", "05", "06"]
# 国リストURL
country_list_url = f"{base_url}CountryList.php?rcode=01"

In [15]:
# 定義済み関数で地域ページ内の国リンクの配列を取得
country_links = fetch_country_links(country_list_url)

# print(country_links)
# print(country_links[0].attrs['href'])

################# 国ごとの繰り返しここから #################
for country in country_links:
    # 個々の要素は "<a href="StationList.php?ccode=10300">アルジェリア</a>" のようになっているので,ここからコード値と国名を抽出
    country_code = country.attrs['href'].split('ccode=')[1]  # "StationList.php?ccode=10300" から コード値のみを抽出
    country_name = country.contents[0] 
    # print(country_code)
    print("######" + country_name + "######")

    # 都市URLの作成
    city_url = f"{base_url}StationList.php?ccode={country_code}"
    # 定義済み関数で国ページ内の国リンクの配列を取得
    city_links = fetch_city_links(city_url)
    # print(city_links)

    ################# 都市ごとの繰り返しここから ##################
    for city in city_links:
        # 個々の要素は "<a href="NrmMonth.php?stn=60560">アインセフラ</a>" のようになっているので,ここからコード値と都市名を抽出
        city_code = city.attrs['href'].split('stn=')[1]
        city_name = city.contents[0]

        print(city_name)

        # データページを取得
        data_url = f"{base_url}NrmMonth.php?stn={city_code}"
        response = requests.get(data_url)
        
        response.encoding = 'UTF-8'
        soup = BeautifulSoup(response.text, 'lxml')   # 標準のhtml.parserだとなぜかtdタグの構造がおかしくなるのでlxmlを使用
        main_div = soup.find(id="main")

        # 位置情報の入ったh3タグを取得し,text形式に変換
        h3_tag = main_div.find('h3')
        h3_text = h3_tag.text

        # 定義済み関数で緯度,経度,高度を取得
        latitude = get_latitude(h3_text)
        longitude = get_longitude(h3_text)
        altitude = get_altitude(h3_text)

        # 気候データの抽出
        data_rows = main_div.find_all('tr')
        # データ構造（例）
        # <tr class="c"><th class="list" style="padding-top:0;padding-bottom:0;">1</th><td align="center" class="list">7.1</td><td align="center" class="list">14.9</td></tr>,

        # データを格納するリスト
        temp_values = []
        rain_values = []
        
        for row in data_rows[2:]:    # 最初の2行はヘッダーなので除外
            cols = row.find_all('td')
            temp = cols[0].text.strip()
            rain = cols[1].text.strip()
            temp_values.append(temp)
            rain_values.append(rain)

        # データフレームに格納する下準備
        # 変数をリストに変換
        city_code_l = [city_code]
        city_name_l = [city_name]
        country_code_l = [country_code]
        country_name_l = [country_name]
        latitude_l = [latitude]
        longitude_l = [longitude]
        altitude_l = [altitude]
        data_url_l = [data_url]

        # 登録データ配列
        new_row = country_code_l + country_name_l + city_code_l + city_name_l + latitude_l + longitude_l + altitude_l + temp_values + rain_values + data_url_l 

        # 作成したデータ配列を,データフレームの最後尾に格納 （len(new_df)で既存配列の「長さ」を返すので,index指定で最後尾となる)
        new_df.loc[len(new_df)] = new_row

        
    
# # 国コードを取得
# country_code = country_links[0].attrs['href'].split('ccode=')[1]
# # 国名を取得
# country_name = country_links[0].contents[0]

# print(country_code)
# print(country_name)

アインセフラ
アドラル
アルジェ
アンナバ
イナメナス
イリジ
インゲザム
インサラー
ウェド
ウームエルブワギー
オラン港
カラ
ガルダイア
クサル
ケンシェラ
ゲルマ
コンスタンティーヌ
ゴレア
サイダ
シェリフ
シディベルアッベス
ジジェル/ACHOUAT
ジジェル港
ジャネ
スキクダ
セティフ
タマンラセット
ティアレット
ティジウーズー
ティベッサ
ティミムン
ティンドゥーフ
テネス
デリス
トゥグルト
トレムセン
ナーマ
ハッシメサウド
バトナ
バヤド
ビスクラ
ブイラ
ブーサアーダ
ベシャル
ベジャイア港
ベジャイア空港
ベニアッベス
ベニサーフ
ボルジブーアレーリジ
マグニア
ムシラ
メシェリア
メデア
モスタガネム
ラグワト
ワルグラ
ルアンダ
アスワン
アリーシュ
アレクサンドリア
キーナ/SOUTH OF VALLEY UNIVERSITY
シワ
ファラーフラ
ヘルワン
マルサマトルーフ
アディスアベバ
アワサ
ゴレ
ゴンダル
ジマ
ディレダワ
デブラマルコス
バヒルダール
ガルワ
ドゥアラ
ンガウンデレ
サル
ポールジャンティ
マコクー
ムイラ
ランバレネ
リーブルビル
アクラ
クマシ
スンヤニ
タコラディ
タマレ
コナクリ/GBESSIA
ビサウ
ボラマ
エルドレット
ガリッサ
ナイロビ
マリンディ
モヤレ
モンバサ
ロドワル
インプフォンド
ウェッソ
シビティ
ジャンバラ
ブラザビル
ポワントノワール
マクア
キンシャサ
アディアケ
アビジャン
オディエンネ
ガニョア
ササンドラ
サンペドロ
タブー
ディンボクロ
ボンドゥク
ヤムスクロ
カサマ
カブウェ
モング
リビングストン
ルンギ
チピンゲ
ハラーレ
ブラワヨ
マスビンゴ
イエロ島
サンタクルスデテネリフェ
テネリフェ島/TENERIFE SUR
フエルテベントゥラ空港
ラスパルマス
ラパルマ島
ランサローテ空港
ロスロデオス空港
セウタ
メリリャ
マンジニ／マツァパ空港
アトバラ
アブーハメッド
アロマ
オベイド
カダーリフ
カッサラ
クライマ
センナール
ドゥンクラ
ニヤラ
ヌフード
ハルツーム
ブールスーダン
ラシャード
ワディハルファ
ワドゥマダニ
セイシェル国際空港
カオラック
ケドゥーグー
コルダ
サンルイ
ジギンショール
タンバクンダ
ダカール
ディウルベル
ポドル
マ

In [18]:
display(new_df)

#出力先ディレクトリの設定
# output_dir = './processed'
output_dir = './processed'

new_df.to_csv(os.path.join(output_dir, 'world_test.csv'), index=False, header=False)


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,country_code,country_name,city_code,city_name,latitude,longitude,altitude,temp_jan,temp_feb,temp_mar,...,rain_apr,rain_may,rain_jun,rain_jul,rain_aug,rain_sep,rain_oct,rain_nov,rain_dec,url
1,10300,アルジェリア,60560,アインセフラ,32.77,-0.6,1058,7.1,9.2,12.9,...,16.9,15.0,6.9,4.1,13.5,21.0,25.8,22.3,9.4,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
2,10300,アルジェリア,60620,アドラル,27.88,-0.18,279,12.9,16.1,20.7,...,2.7,0.5,0.2,0.0,0.6,3.5,0.5,0.6,0.7,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
3,10300,アルジェリア,60369,アルジェ,36.77,3.1,9,14.1,14.0,15.8,...,60.9,38.8,12.5,1.6,7.1,33.4,58.4,137.8,105.7,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
4,10300,アルジェリア,60360,アンナバ,36.83,7.82,3,11.2,11.2,13.2,...,53.4,36.2,10.5,2.4,10.8,38.8,64.7,96.8,101.9,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,14800,南アフリカ,68341,リヒテンバーグ,-26.13,26.17,1485,22.4,22.1,20.9,...,-,-,-,-,-,-,-,-,-,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
338,14800,南アフリカ,68723,レイングズバーグ,-33.2,20.87,655,25.9,26.1,23.8,...,-,-,-,-,-,-,-,-,-,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
339,14800,南アフリカ,68718,ロバートソン,-33.79,19.88,203,23.6,23.9,22.4,...,-,-,-,-,-,-,-,-,-,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
340,10140,西サハラ,60096,ダフラ,23.71,-15.93,-,18.2,18.5,19.2,...,0.7,1.9,0.0,0.2,1.4,5.7,2.6,0.4,1.6,https://www.data.jma.go.jp/cpd/monitor/nrmlist...


In [19]:

################# ここから試作段階のコード ################################


# 国のページをリクエスト
city_url = f"{base_url}StationList.php?ccode={country_code}"
response = requests.get(city_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
# 都市リストを取得
city_links = soup.find_all(href=re.compile("NrmMonth.php"))
print(city_links)
print(len(city_links))


print(city_links[0].attrs['href'])

# 都市コードを取得
city_code = city_links[0].attrs['href'].split('stn=')[1]
# 都市名を取得
city_name = city_links[0].contents[0]

print(city_code)
print(city_name)

[<a href="NrmMonth.php?stn=60560">アインセフラ</a>, <a href="NrmMonth.php?stn=60620">アドラル</a>, <a href="NrmMonth.php?stn=60369">アルジェ</a>, <a href="NrmMonth.php?stn=60360">アンナバ</a>, <a href="NrmMonth.php?stn=60611">イナメナス</a>, <a href="NrmMonth.php?stn=60640">イリジ</a>, <a href="NrmMonth.php?stn=60690">インゲザム</a>, <a href="NrmMonth.php?stn=60630">インサラー</a>, <a href="NrmMonth.php?stn=60559">ウェド</a>, <a href="NrmMonth.php?stn=60421">ウームエルブワギー</a>, <a href="NrmMonth.php?stn=60461">オラン港</a>, <a href="NrmMonth.php?stn=60367">カラ</a>, <a href="NrmMonth.php?stn=60566">ガルダイア</a>, <a href="NrmMonth.php?stn=60514">クサル</a>, <a href="NrmMonth.php?stn=60476">ケンシェラ</a>, <a href="NrmMonth.php?stn=60403">ゲルマ</a>, <a href="NrmMonth.php?stn=60419">コンスタンティーヌ</a>, <a href="NrmMonth.php?stn=60590">ゴレア</a>, <a href="NrmMonth.php?stn=60536">サイダ</a>, <a href="NrmMonth.php?stn=60425">シェリフ</a>, <a href="NrmMonth.php?stn=60520">シディベルアッベス</a>, <a href="NrmMonth.php?stn=60351">ジジェル/ACHOUAT</a>, <a href="NrmMonth.php?stn=60353

In [9]:
# データページを取得
data_url = f"{base_url}NrmMonth.php?stn={city_code}"
# data_url = f"{base_url}NrmMonth.php?stn=64820"
response = requests.get(data_url)

response.encoding = 'UTF-8'
soup = BeautifulSoup(response.text, 'lxml')   # 標準のhtml.parserだとなぜかtdタグの構造がおかしくなるのでlxmlを使用

In [10]:
# print(data_url)
print(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "https://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ja" xml:lang="ja" xmlns="https://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>気象庁｜平年値データ</title>
<meta content="気象庁 Japan Meteorological Agency" name="Author"/>
<meta content="気象庁 Japan Meteorological Agency" name="keywords"/>
<meta content="気象庁が提供するページです" name="description"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<link href="/com/css/define.css" media="all" rel="stylesheet" type="text/css"/>
<script src="/com/js/jquery.js" type="text/javascript"></script>
<style media="all" type="text/css"><!-- @import url(/com/default.css); --></style>
</head>
<body>
<noscript><div>すべての機能をご利用いただくにはJavascriptを有効にしてください。</div></noscript>
<!--header start -->
<div id="header_area">
<div id="nheader">
<div id="logo"><a href=

In [11]:
# データ格納
main_div = soup.find(id="main")

# print(main_div)

# 緯度
h3_tag = main_div.find('h3')
print(h3_tag)

h3_text = h3_tag.text


# 緯度を抽出
lat_pattern = r"緯度：([\d.]+)°([NS])"
lat_match = re.search(lat_pattern, h3_text)
print(lat_match)
if lat_match:
    lat_value = float(lat_match.group(1))   # キャプチャグループ1は ([\d.]+) 部分
    lat_direction = lat_match.group(2)      # キャプチャグループ2は ([NS]) 部分
    if lat_direction == 'S':                # 南半球Sなら値を負数に
        latitude = -lat_value
    else:
        latitude = lat_value

print(latitude)

# 経度を抽出
lon_pattern = r"経度：([\d.]+)°([EW])"
lon_match = re.search(lon_pattern, h3_text)
if lon_match:
    lon_value = float(lon_match.group(1))
    lon_direction = lon_match.group(2)
    if lon_direction == 'W':
        longitude = -lon_value
    else:
        longitude = lon_value

print(longitude)


# 高度を抽出
alt_pattern = r"高度：(\d+) \(m\)"
alt_match = re.search(alt_pattern, h3_text)
if alt_match:
    altitude = int(alt_match.group(1))
print(altitude)



<h3>アインセフラ - アルジェリア   緯度：32.77°N   経度：0.60°W   高度：1058 (m)</h3>
<re.Match object; span=(18, 28), match='緯度：32.77°N'>
32.77
-0.6
1058


In [54]:
# データの抽出

data_rows = main_div.find_all('tr')
print(data_rows)

# row = data_rows[2]
# print(row)
# cols = row.find_all("td")
# print(cols[0].contents[0])


# データを格納するリスト
temp_values = []
rain_values = []

for row in data_rows[2:]:    # 最初の2行はヘッダーなので除外
    cols = row.find_all('td')
    temp = cols[0].text.strip()
    rain = cols[1].text.strip()
    temp_values.append(temp)
    rain_values.append(rain)

print(temp_values)
print(rain_values)
    

[<tr>
<th class="list" rowspan="2">月</th>
<th class="list" colspan="2">平年値</th>
</tr>, <tr class="list">
<th class="list">月平均気温<br/>℃</th>
<th class="list">月降水量<br/>mm</th>
</tr>, <tr class="c">
<th class="list" style="padding-top:0;padding-bottom:0;">1</th>
<td align="center" class="list">7.1  
</td><td align="center" class="list">14.9  
</td></tr>, <tr class="c">
<th class="list" style="padding-top:0;padding-bottom:0;">2</th>
<td align="center" class="list">9.2  
</td><td align="center" class="list">11.2  
</td></tr>, <tr class="c">
<th class="list" style="padding-top:0;padding-bottom:0;">3</th>
<td align="center" class="list">12.9  
</td><td align="center" class="list">15.9  
</td></tr>, <tr class="c">
<th class="list" style="padding-top:0;padding-bottom:0;">4</th>
<td align="center" class="list">16.8  
</td><td align="center" class="list">16.9  
</td></tr>, <tr class="c">
<th class="list" style="padding-top:0;padding-bottom:0;">5</th>
<td align="center" class="list">21.5  
</td><td

In [29]:
# 最終出力データフレーム
new_df = pd.DataFrame(columns=range(0,32))

# ヘッダーを追加
header = ['country_code', 'country_name', 'city_code', 'city_name',  
          'latitude', 'longitude', 'altitude',
          'temp_jan', 'temp_feb', 'temp_mar', 'temp_apr', 'temp_may', 'temp_jun',
          'temp_jul', 'temp_aug', 'temp_sep', 'temp_oct', 'temp_nov', 'temp_dec',
          'rain_jan', 'rain_feb', 'rain_mar', 'rain_apr', 'rain_may', 'rain_jun',
          'rain_jul', 'rain_aug', 'rain_sep', 'rain_oct', 'rain_nov', 'rain_dec',
          'url']
new_df.loc[0] = header

display(new_df)

# 変数をリストに変換
city_code_l = [city_code]
city_name_l = [city_name]
country_code_l = [country_code]
country_name_l = [country_name]
latitude_l = [latitude]
longitude_l = [longitude]
altitude_l = [altitude]
data_url_l = [data_url]

print(data_url_l)

new_row = country_code_l + country_name_l + city_code_l + city_name_l + latitude_l + longitude_l + altitude_l + temp_values + rain_values + data_url_l 

print(new_row)

# 作成したデータ配列を,データフレームの最後尾に格納（len(new_df)で既存配列の「長さ」を返すので,index指定で最後尾となる)
new_df.loc[len(new_df)] = new_row

display(new_df)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,country_code,country_name,city_code,city_name,latitude,longitude,altitude,temp_jan,temp_feb,temp_mar,...,rain_apr,rain_may,rain_jun,rain_jul,rain_aug,rain_sep,rain_oct,rain_nov,rain_dec,url


['https://www.data.jma.go.jp/cpd/monitor/nrmlist/NrmMonth.php?stn=60560']
['10300', 'アルジェリア', '60560', 'アインセフラ', 32.77, -0.6, 1058, '7.1', '9.2', '12.9', '16.8', '21.5', '26.7', '31.0', '29.5', '24.4', '18.6', '12.0', '8.2', '14.9', '11.2', '15.9', '16.9', '15.0', '6.9', '4.1', '13.5', '21.0', '25.8', '22.3', '9.4', 'https://www.data.jma.go.jp/cpd/monitor/nrmlist/NrmMonth.php?stn=60560']


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,country_code,country_name,city_code,city_name,latitude,longitude,altitude,temp_jan,temp_feb,temp_mar,...,rain_apr,rain_may,rain_jun,rain_jul,rain_aug,rain_sep,rain_oct,rain_nov,rain_dec,url
1,10300,アルジェリア,60560,アインセフラ,32.77,-0.6,1058,7.1,9.2,12.9,...,16.9,15.0,6.9,4.1,13.5,21.0,25.8,22.3,9.4,https://www.data.jma.go.jp/cpd/monitor/nrmlist...
